In [1]:
from fedn.utils.helpers.helpers import get_helper
from ultralytics import YOLO
import torch
import collections
import numpy as np

HELPER_MODULE = "numpyhelper"
helper = get_helper(HELPER_MODULE)

def get_best_device():
    """ Get the best device available.

    :return: The best device available.
    :rtype: str
    """
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"
    return device

def compile_model():
    """Compile the YOLO model.

    :return: The compiled YOLO model.
    :rtype: torch.nn.Module
    """
    device = get_best_device()
    return YOLO("model.yaml").to(device)

def save_parameters(model, out_path):
    """Save model parameters to file.

    :param model: The model to serialize.
    :type model: torch.nn.Module
    :param out_path: The path to save to.
    :type out_path: str
    """
    parameters_np = [
    val.cpu().numpy().astype(np.float32) if val.dtype.is_floating_point else val.cpu().numpy()
    for _, val in model.model.model.state_dict().items()
    ]
    helper.save(parameters_np, out_path)

def load_parameters(model_path):
    """Load model parameters from a file.

    :param model_path: The path to the model parameters file.
    :type model_path: str
    :return: The YOLO model with loaded parameters.
    :rtype: YOLO
    """
    parameters_np = helper.load(model_path)

    yolo_model = compile_model()
    torch_model = yolo_model.model.model

    keys = list(torch_model.state_dict().keys())
    if len(parameters_np) != len(keys):
        raise ValueError(f"Mismatch: {len(parameters_np)} parameters vs {len(keys)} model keys")

    state_dict = collections.OrderedDict({
        key: torch.tensor(val) for key, val in zip(keys, parameters_np)
    })
    torch_model.load_state_dict(state_dict, strict=False)

    yolo_model.ckpt = {"model": torch_model}
    return yolo_model

In [2]:
model = load_parameters("weights/face_finder_best.npz")

WARNING ⚠️ no model scale passed. Assuming scale='n'.


In [4]:
params = {
    'data': '/hopsfs/Jupyter/yolov8-face/data/widerface.yaml',
    'epochs': 1,
    'batch': 32,
    'imgsz': 640,
    'device': 0,
    'resume': False,
    'workers': 0,
    'cache': "ram",
    'amp': True,
}

print(params)

{'data': '/hopsfs/Jupyter/yolov8-face/data/widerface.yaml', 'epochs': 1, 'batch': 32, 'imgsz': 640, 'device': 0, 'resume': False, 'workers': 0, 'cache': 'ram', 'amp': True}


In [5]:
model.train(**params)

New https://pypi.org/project/ultralytics/8.3.182 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.181 🚀 Python-3.10.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/hopsfs/Jupyter/yolov8-face/data/widerface.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=model.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train21, nbs=64, nms=False, op

train: Scanning /hopsfs/Jupyter/yolov8-face/data/widerface/train... 2464 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2464/2464 [00:47<00:00, 51.58it/s]


train: New cache created: /hopsfs/Jupyter/yolov8-face/data/widerface/train.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (2.0GB RAM): 100%|██████████| 2464/2464 [00:21<00:00, 114.21it/s]


val: Fast image access ✅ (ping: 0.1±0.0 ms, read: 8.6±5.7 MB/s, size: 90.7 KB)


...

        1/1      18.7G      1.417     0.7259      1.031        570        640: 100%|██████████| 77/77 [00:24<00:00,  3.09it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  2.69it/s]


                   all        617       7891      0.847      0.572      0.656      0.365

1 epochs completed in 0.009 hours.
Optimizer stripped from runs/detect/train21/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train21/weights/best.pt, 6.2MB

Validating runs/detect/train21/weights/best.pt...
Ultralytics 8.3.181 🚀 Python-3.10.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  3.07it/s]


                   all        617       7891      0.847      0.572      0.657      0.364
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train21


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f609935e3e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [7]:
import hopsworks
mr = hopsworks.login().get_model_registry()

2025-08-21 09:13:34,055 INFO: Python Engine initialized.

Logged in to project, explore it here https://snurran.devnet.hops.works/p/120


In [8]:
import os
model_dir = "mr_model"
os.makedirs(f"{model_dir}/images", exist_ok=True)

In [9]:
save_parameters(model, f"./{model_dir}/fine-tuned-model.npz")

### Model Evaluation
Predict bounding boxes for an example image and save the output image with the model.

In [10]:
from PIL import Image
img_path = "data/images/bus.jpg"
results = model.predict(
    img_path,
    imgsz=640,
    conf=0.75,
    iou=0.7,
    device=0,
    verbose=False
)

img = results[0].plot()  # BGR numpy array
img = Image.fromarray(img[..., ::-1])  # Convert to RGB for PIL

base, _ = os.path.splitext(os.path.basename(img_path))
output_filename = f"./{model_dir}/images/{base}-faces-detected.png"
output_path = os.path.abspath(output_filename)
img.save(output_path, format="PNG")

### Save Trained Model to Hopsworks Model Registry

Save the serialized model, evaluation images, and any metrics to the model registry

In [11]:
metrics = {
    "epochs": params['epochs'],
    "batch": params['batch'],    
}

faces_model = mr.python.create_model(
    name="facerecognition", 
    metrics=metrics,
    description="Yolo-v8 face recognition model", 
)

# Save the model to the specified directory
faces_model.save(model_dir)

2025-08-21 09:13:38,503 WARNING: ProvenanceWarning: Model schema cannot not be inferred without both the feature view and the training dataset version.



  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://snurran.devnet.hops.works/p/120/models/facerecognition/5


Model(name: 'facerecognition', version: 5)